# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Import what you need

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd
import time

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [2]:
driver = webdriver.Chrome()
driver.get('https://jportal.mdcourts.gov/license/index_disclaimer.jsp')

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages, or you can close and re-open Chrome.

- Check the checkbox, then submit the form to accept their terms of service

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

- *TIP: if something doesn't have anything special about it, xpath might be your best bet*

In [3]:
# click the checkbox
driver.find_element_by_id('checkbox').click()
# wait a second
time.sleep(1)
# click the search
driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]').click()


Now click the **Search License Records** link up top in the navigation to get to the search page.

- *TIP: Honestly you could also just visit the URL directly now since you filled out that terms of service thing*

In [4]:
# click search license records
driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[6]/td[2]/a[2]').click()

### Perform your search

Pick "Statewide" for the jurisdiction dropdown, and `VAP%` into the Trade Name field. The `%` is a wildcard.

In [5]:
dropdown = Select (driver.find_element_by_id('slcJurisdiction'))
dropdown.select_by_visible_text('Statewide')

time.sleep(1)

driver.find_element_by_id('txtTradeName').send_keys('Vap%')
time.sleep(1)


And now, of course, **submit the form**.

- *TIP: Since scrolling to buttons can be a pain, sometimes it's easier to select the form and use `.submit()` instead of `.click()`ing the button*

In [6]:

driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]').submit()

## (Try to) scrape the results

Let's start by just **printing this stuff**. We'll save it as a dataframe later on.

For now, just scrape **each store's name**, then cry a little. Fact: this is an impossible and miserable page. 

In [7]:
names = driver.find_elements_by_class_name('searchlistitem')
for name in names:
    print(name.text)

VAPE IT STORE II
VAPE IT STORE I
VAPEPAD THE
VAPE FROG
VAPE FROG


To avoid struggling with the search results page, we're going to use the **detail page** instead. Try to figure out how to select it and click it inside of your `for` loop.

- *TIP: Instead of just looking for an `a` or an `img`, you might want to look for one of its parents first, then click. This might affect the way you print out the shop's name, too*
- *TIP: Not all of them have links! You can wrap in try/except to skip it, or you can check to see if the shop's status is Pending.*

In [8]:
# use the count to click in the for loop

titles = driver.find_elements_by_class_name('searchfieldtitle')
for title in titles:
    try:
        detail_page = title.find_elements_by_tag_name('td')[2].find_element_by_tag_name('a')
        name = title.find_elements_by_tag_name('td')[1]
        #detail_pages.click()
        print(name.text," :",detail_page.get_attribute('href'))
        #driver.back()
        time.sleep(1)
    except:
         print(name.text,":",'no link')

# will creare a dataframe with name and url to click later

VAPE IT STORE II  : https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=fBGMag8qjYc%3D
VAPE IT STORE I  : https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=D%2BKOBQdQB3c%3D
VAPEPAD THE  : https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=6qqVFBESa5U%3D
VAPEPAD THE : no link
VAPEPAD THE : no link


Okay, now let's get to action. For each result, **click the link to the detail page** and print out the following information:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

If it doesn't have a detail page, just print out the name and that's all we need.

- *TIP: When you're done getting the information, you probably want to click back to the search results*
- *TIP: You might enjoy `find_element_by_partial_link_text` to do that*
- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*

> **IMPORTANT NOTE:** This is doomed. It's useful to do, but your current process is doomed. Once you get a `stale element reference` error move on to the next cell.

In [9]:
mailing = driver.find_elements_by_class_name('tablecelltext')[0].find_element_by_tag_name('td')
print("mailing address:", mailing.text)
print("--------------")

location= driver.find_elements_by_class_name('tablecelltext')[1].find_element_by_tag_name('td')
print("location address:",location.text)
print("--------------")

license_info = driver.find_elements_by_class_name('tablecelltext')[0].find_elements_by_tag_name('td')[1]
print("license info:",license_info.text)

paid =driver.find_elements_by_class_name('tablecelltext')[7].find_elements_by_tag_name('td')[1]
print("total amount paid:",paid.text)
print("--------------")

issue =driver.find_elements_by_class_name('tablecelltext')[8].find_element_by_tag_name('td')
print("issued by:",issue.text)
print("--------------")

licenses = driver.find_elements_by_tag_name('table')[-4]
license_1= licenses.find_elements_by_class_name('tablecelltext')[0].find_elements_by_tag_name('td')[2].text
license_2 =licenses.find_elements_by_class_name('tablecelltext')[1].find_elements_by_tag_name('td')[2].text
license_3 =licenses.find_elements_by_class_name('tablecelltext')[2].find_elements_by_tag_name('td')[2].text
print(license_1,";", license_2,";",license_3)

mailing address:  
--------------
location address:  
--------------
license info: AMIN NARGIS
total amount paid: Wicomico County
--------------
issued by:  
--------------
Lic. Status: Issued ; License: 22375606 ; Issued Date: 4/27/2018


In [10]:
# try out licenses
driver.find_elements_by_tag_name('table')[-4].find_elements_by_class_name('tablecelltext')[0].find_elements_by_tag_name('td')[2].text
driver.find_elements_by_tag_name('table')[-4].find_elements_by_class_name('tablecelltext')[1].find_elements_by_tag_name('td')[2].text

'License: 22375606'

### Stale message reference

Once you navigate away from a page, and you go back to it, you can't use the variables from the first time you were on the page. So, we got a list of results when we first visited, clicked to the details page, clicked back, and now our original list is "stale."

This is sad.

Let's try this again: loop through the results and create a dataframe with `name` and `url` columns. And yes, some of them won't have URLs.

In [11]:
mylist = []

titles = driver.find_elements_by_class_name('searchfieldtitle')
for title in titles:
    data ={}
    data['name'] = title.find_elements_by_tag_name('td')[1].text
    try:
        data['url']= title.find_element_by_tag_name('a').get_attribute('href')

    except:
        data['url']= 'NaN'
    mylist.append(data)
print(mylist)

df = pd.DataFrame(mylist)
df.head()


[{'name': 'VAPE IT STORE II', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=fBGMag8qjYc%3D'}, {'name': 'VAPE IT STORE I', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=D%2BKOBQdQB3c%3D'}, {'name': 'VAPEPAD THE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=6qqVFBESa5U%3D'}, {'name': 'VAPE FROG', 'url': 'NaN'}, {'name': 'VAPE FROG', 'url': 'NaN'}]


,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,VAPE FROG,NaN
4,VAPE FROG,NaN


### Getting all of the results

After you've looped through the results on one page, we're going to want to go to the next page! Add a line to make it click the 'Next' button down at the bottom

- *Tip: `find_element_by_partial_link_text` will be your friend*
- *Tip: You might need to do the scrolling thing to get it onto the screen (and by that I mean, you WILL need to, so you should)*

Confirm that it moves to the next page (it doesn't need to scrape anything yet)

In [12]:
next_button = driver.find_element_by_partial_link_text('Next')
driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
next_button.click()

### Wrapping with `while`

> Go back to the first page of results before you try to run this

You have a bunch of scraping code. It clicks the next button, then it stops. But you'd like it to go back up to the top! You can make that happen with a special `while` loop.

```python
while True:
    # Scrape your stuff
    # Click next button
```

This will go on FOREVER AND EVER until there is an error (when it can't find the Next button on the last page of results, you'll get an error).

- *Tip: Print out "Scraping a new page" every time you visit a new page, just to check that it's working*

In [13]:
while True:
    mylist = []
    titles = driver.find_elements_by_class_name('searchfieldtitle')
    for title in titles:
        data ={}
        data['name'] = title.find_elements_by_tag_name('td')[1].text
        try:
            data['url']= title.find_element_by_tag_name('a').get_attribute('href')
        except:
            data['url']= 'NaN'
        mylist.append(data)

    next_button = driver.find_element_by_partial_link_text('Next')
    driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
    next_button.click()

# Yes, it gets all 7 pages 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Next"}
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.3 x86_64)


### Making it perfect

> Go back to the first page of results before you try to run this

Wrap all of your code in a `try`/`except` so that it doesn't finish with an error and you'll be good to go.

**Confirm your list has all of the vape shops in it.** If not, check where you are creating your empty list (`[]`) - if you do it in the wrong spot, it will overwrite your list every time you visit a page.

In [16]:
mylist=[]
try:
    while True:
        
        titles = driver.find_elements_by_class_name('searchfieldtitle')
        for title in titles:
            data ={}
            data['name'] = title.find_elements_by_tag_name('td')[1].text
            try:
                data['url']= title.find_element_by_tag_name('a').get_attribute('href')
            except:
                data['url']= 'NaN'
            mylist.append(data)
        print(mylist)

        next_button = driver.find_element_by_partial_link_text('Next')
        driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
        next_button.click()
except:
    pass 



[{'name': 'VAPE IT STORE II', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=fBGMag8qjYc%3D'}, {'name': 'VAPE IT STORE I', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=D%2BKOBQdQB3c%3D'}, {'name': 'VAPEPAD THE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=6qqVFBESa5U%3D'}, {'name': 'VAPE FROG', 'url': 'NaN'}, {'name': 'VAPE FROG', 'url': 'NaN'}]
[{'name': 'VAPE IT STORE II', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=fBGMag8qjYc%3D'}, {'name': 'VAPE IT STORE I', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=D%2BKOBQdQB3c%3D'}, {'name': 'VAPEPAD THE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=6qqVFBESa5U%3D'}, {'name': 'VAPE FROG', 'url': 'NaN'}, {'name': 'VAPE FROG', 'url': 'NaN'}, {'name': 'VAPE LOFT THE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=%2F4hH9a%2B3mEo%3D'}, {'name': 'VAPE N CIGAR', 'url': 'https://jpor

[{'name': 'VAPE IT STORE II', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=fBGMag8qjYc%3D'}, {'name': 'VAPE IT STORE I', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=D%2BKOBQdQB3c%3D'}, {'name': 'VAPEPAD THE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=6qqVFBESa5U%3D'}, {'name': 'VAPE FROG', 'url': 'NaN'}, {'name': 'VAPE FROG', 'url': 'NaN'}, {'name': 'VAPE LOFT THE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=%2F4hH9a%2B3mEo%3D'}, {'name': 'VAPE N CIGAR', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=Hdbg6Tf6nbY%3D'}, {'name': 'VAPE DOJO', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=Nma7ehpVqsw%3D'}, {'name': 'VAPE HAVEN', 'url': 'NaN'}, {'name': 'VAPORS LOUNGE', 'url': 'NaN'}, {'name': 'VAPORFI SOUTHSIDE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=b47SKm89xB4%3D'}, {'name': 'VAPE DO JO', 'url': 'https://jportal.

### Save this data as a csv

The filename should be `vape-shops-basic.csv`.

In [17]:
df = pd.DataFrame(mylist)
df

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,VAPE FROG,NaN
4,VAPE FROG,NaN
5,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicense...
6,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicense...
7,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicense...
8,VAPE HAVEN,NaN
9,VAPORS LOUNGE,NaN


In [18]:
df.to_csv('vape-shops-basic.csv',index=False)

In [20]:
df_basic=pd.read_csv('vape-shops-basic.csv')
df_basic

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,VAPE FROG,NaN
4,VAPE FROG,NaN
5,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicense...
6,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicense...
7,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicense...
8,VAPE HAVEN,NaN
9,VAPORS LOUNGE,NaN


# Okay, let's scrape!

All right, get the actual data!

### Look at the URL of your first row

- *TIP: Remember `pd.set_option('display.max_colwidth', -1)` will let you see alllll of your strings*

In [21]:
pd.set_option('display.max_colwidth',-1)


In [22]:
df_basic.url[0]

'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=fBGMag8qjYc%3D'

### Use Selenium to visit that page

In [23]:
driver.get(df_basic.url[0])

### Now, just like you did before, grab the additional data

You should probably save it into a dictionary! Don't try to put it into the dataframe yet, though. You want:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

.

- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*
- *TIP: If you've gotten addicted to xpath, total amount paid and issued by might not work with it when doing other shops. You'll want to test it!*

In [24]:
mydic={}
mydic['mailing'] = driver.find_elements_by_class_name('tablecelltext')[0].find_element_by_tag_name('td').text

mydic['location']= driver.find_elements_by_class_name('tablecelltext')[1].find_element_by_tag_name('td').text

mydic['license_info'] = driver.find_elements_by_class_name('tablecelltext')[0].find_elements_by_tag_name('td')[1].text

mydic['total amount paid'] =driver.find_elements_by_class_name('tablecelltext')[7].find_elements_by_tag_name('td')[1].text

mydic['issued by'] =driver.find_elements_by_class_name('tablecelltext')[8].find_element_by_tag_name('td').text

# mydic['licenese']=[license_1,license_2,license_3]
# licenses_ = driver.find_elements_by_tag_name('table')[-4]
# license_1= licenses.find_elements_by_class_name('tablecelltext')[0].find_elements_by_tag_name('td')[2].text
# license_2 =licenses.find_elements_by_class_name('tablecelltext')[1].find_elements_by_tag_name('td')[2].text
# license_3 =licenses.find_elements_by_class_name('tablecelltext')[2].find_elements_by_tag_name('td')[2].text


mydic

{'mailing': 'AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804',
 'location': 'VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801',
 'license_info': 'License Status: Issued\nLicense No.: 22375606\nControl No.: 22884439\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury',
 'total amount paid': '$ 41.00',
 'issued by': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198'}

### Move all of this into one cell

It should visit the URL, then grab the data and put it into a dictionary.

In [25]:
driver.get(df_basic.url[0])
mydic={}
mydic['mailing'] = driver.find_elements_by_class_name('tablecelltext')[0].find_element_by_tag_name('td').text

mydic['location']= driver.find_elements_by_class_name('tablecelltext')[1].find_element_by_tag_name('td').text

mydic['license_info'] = driver.find_elements_by_class_name('tablecelltext')[0].find_elements_by_tag_name('td')[1].text

mydic['total amount paid'] =driver.find_elements_by_class_name('tablecelltext')[7].find_elements_by_tag_name('td')[1].text

mydic['issued by'] =driver.find_elements_by_class_name('tablecelltext')[8].find_element_by_tag_name('td').text

# mydic['licenese']=[license_1,license_2,license_3]
# licenses_ = driver.find_elements_by_tag_name('table')[-4]
# license_1= licenses.find_elements_by_class_name('tablecelltext')[0].find_elements_by_tag_name('td')[2].text
# license_2 =licenses.find_elements_by_class_name('tablecelltext')[1].find_elements_by_tag_name('td')[2].text
# license_3 =licenses.find_elements_by_class_name('tablecelltext')[2].find_elements_by_tag_name('td')[2].text


mydic

{'mailing': 'AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804',
 'location': 'VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801',
 'license_info': 'License Status: Issued\nLicense No.: 22375606\nControl No.: 22884439\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury',
 'total amount paid': '$ 41.00',
 'issued by': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198'}

### Change it into a function

You'll want to have this function accept a `row`, and send back a `pd.Series`. You can just use `pd.Series(your_dictionary)` (but it better have a better name than `your_dictionary`!).

- *TIP: Make sure you `return` something!*
- *TIP: Make sure you change everything to reflect the row's url, not the URL you typed in*

In [29]:
# use -3 and -2 for total amout paid and issued by
def get_license_detail(row):
    try:
        driver.get(row['url'])
        mydic={}
        mydic['mailing'] = driver.find_elements_by_class_name('tablecelltext')[0].find_element_by_tag_name('td').text

        mydic['location']= driver.find_elements_by_class_name('tablecelltext')[1].find_element_by_tag_name('td').text

        mydic['license_info'] = driver.find_elements_by_class_name('tablecelltext')[0].find_elements_by_tag_name('td')[1].text

        mydic['total amount paid'] =driver.find_elements_by_class_name('tablecelltext')[-3].find_elements_by_tag_name('td')[1].text

        mydic['issued by'] =driver.find_elements_by_class_name('tablecelltext')[-2].find_element_by_tag_name('td').text


        return pd.Series(mydic)
    except:
        return pd.Series({})


### Use your dataframe and `.apply` to pull all of the data from the vape shops

Once you know it's working, use the whole 

- *TIP: Try using it with `.head(3)` first*
- *TIP: You'll want to use `.apply` with your new function*
- *TIP: Issued By and Total Paid are goign to give you problems if you tried to use xpath! Try checking the classes and think about find_elementSSSSS and working backwards instead of forwards.*
- *TIP: You might need a `try`/`except`*
- *TIP: Make sure you're using `axis=1`*
- *TIP: Use `.join` the big thing with all of the `dfs` - make sure you name them right!*

In [ ]:
df_basic

In [33]:
# I checked it with my previous homework and it should be working!
# I don't know why I have error here

df_full_partial = df_basic.head(3).apply(get_license_detail,axis=1).join(df_basic)
df_full_partial

,mailing,location,license_info,total amount paid,issued by,name,url
0,"AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804","VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801",License Status: Issued\nLicense No.: 22375606\nControl No.: 22884439\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury,$ 41.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198",VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=fBGMag8qjYc%3D
1,"AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804","VAPE IT STORE I\n1724 N SALISBURY BLVD UNIT 2\nSALISBURY, MD 21801",License Status: Issued\nLicense No.: 22375605\nControl No.: 22591855\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury,$ 24.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198",VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=D%2BKOBQdQB3c%3D
2,"ANJ DISTRIBUTIONS LLC\n2504 ORCHARD KNOLL WAY\nODENTON, MD 21113","VAPEPAD THE\n2299 JOHNS HOPKINS ROAD\nGAMBRILLS, MD 21054",License Status: Issued\nLicense No.: 02304705\nControl No.: 02685930\nDate of Issue: 4/05/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 02 Anne Arundel County,$ 94.00,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401",VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=6qqVFBESa5U%3D


In [32]:
df_full = df_basic.apply(get_license_detail,axis=1).join(df_basic)
df_full

,issued by,license_info,location,mailing,total amount paid,name,url
0,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198",License Status: Issued\nLicense No.: 22375606\nControl No.: 22884439\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury,"VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801","AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804",$ 41.00,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=fBGMag8qjYc%3D
1,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198",License Status: Issued\nLicense No.: 22375605\nControl No.: 22591855\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury,"VAPE IT STORE I\n1724 N SALISBURY BLVD UNIT 2\nSALISBURY, MD 21801","AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804",$ 24.00,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=D%2BKOBQdQB3c%3D
2,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401",License Status: Issued\nLicense No.: 02304705\nControl No.: 02685930\nDate of Issue: 4/05/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 02 Anne Arundel County,"VAPEPAD THE\n2299 JOHNS HOPKINS ROAD\nGAMBRILLS, MD 21054","ANJ DISTRIBUTIONS LLC\n2504 ORCHARD KNOLL WAY\nODENTON, MD 21113",$ 94.00,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=6qqVFBESa5U%3D
3,NaN,NaN,NaN,NaN,NaN,VAPE FROG,NaN
4,NaN,NaN,NaN,NaN,NaN,VAPE FROG,NaN
5,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401",License Status: Issued\nLicense No.: 02310799\nControl No.: 02686069\nDate of Issue: 4/03/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 02 Anne Arundel County,"VAPE LOFT THE\n185 MITCHELLS CHANCE RD\nEDGEWATER, MD 21037",DISBROW II EMERSON HARRINGTON,$ 154.00,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=%2F4hH9a%2B3mEo%3D
6,"WAYNE A. ROBEY, CLERK OF CIRCUIT COURT\n9250 BENDIX ROAD\nCOLUMBIA, MARYLAND 21045",License Status: Issued\nLicense No.: 13343011\nControl No.: 13856368\nDate of Issue: 4/30/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 13 Howard County,"VAPE N CIGAR\n7104 MINSTREL UNIT #7\nCOLUMBIA, MD 21045",DISCOUNT TOBACCO ESSEX LLC,$ 84.00,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=Hdbg6Tf6nbY%3D
7,"DONALD B. SEALING II, CLERK OF CIRCUIT COURT\n55 NORTH COURT STREET\nWESTMINSTER, MARYLAND 21157-5155",License Status: Issued\nLicense No.: 06327188\nControl No.: 06946760\nDate of Issue: 4/05/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 06 Westminster,"VAPE DOJO\n330 ONE FORTY VILLAGE ROAD\nUNIT 15\nWESTMINSTER, MD 21157",FAIRGROUND VILLAGE LLC,$ 179.00,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=Nma7ehpVqsw%3D
8,NaN,NaN,NaN,NaN,NaN,VAPE HAVEN,NaN
9,NaN,NaN,NaN,NaN,NaN,VAPORS LOUNGE,NaN


## Save as `vape-total.csv`

Make sure you don't save the index! Open it up in a text editor or Excel to make sure it's correct.

In [34]:
df_full.to_csv("vape-total.csv", index = False)